# Time Series Prediction using LSTM in Python

After the first approach in the previous file, we are now trying to set up the whole process more formally.

### 0.1 Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import warnings
warnings.filterwarnings('ignore')

## 1. Functions

### 1.1 Loading

We need to handle the following:

* NA Values: Ensure that 'NA' strings are correctly interpreted as NaN.
* Date Parsing: Parse the 'time' column as dates.

In [2]:
def load_data(file_path):
    # Read the CSV file, specifying NA values and parsing dates
    df = pd.read_csv(
        file_path,
        na_values=['NA'],
        parse_dates=['time'],
        infer_datetime_format=True
    )
    return df

* `na_values=['NA']`: Interprets 'NA' strings as NaN.
* `parse_dates=['time']`: Parses the 'time' column as datetime objects.
* `infer_datetime_format`=True: Speeds up date parsing by inferring the format.

### 1.2 Data Preprocessing

In [3]:

def scale2(x):
    return (x - np.nanmean(x)) / np.nanstd(x)

def clean_data(df):
    # Keep relevant columns
    df = df[['geo', 'time', 'values']].copy()
    
    # Sort by time
    df = df.sort_values('time').reset_index(drop=True)
    
    # Scaling and log transformation
    df['values_scale'] = df.groupby('geo')['values'].transform(scale2)
    df['values_log'] = np.log(df['values'])
    df['values_scale_mean'] = df.groupby('geo')['values'].transform('mean')
    df['values_scale_sd'] = df.groupby('geo')['values'].transform('std')
    
    return df

### 1.3. Helpers

In [4]:

# 3. Helper Functions for LSTM
def create_sequences(values, n_steps=12, n_future=2):
    X, y = [], []
    for i in range(len(values) - n_steps - n_future + 1):
        X.append(values[i:i+n_steps])
        y.append(values[i+n_steps:i+n_steps+n_future])
    X = np.array(X).reshape(-1, n_steps, 1)
    y = np.array(y)
    return X, y

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=input_shape))
    model.add(Dense(2))  # Predicting 2 steps ahead
    model.compile(optimizer='adam', loss='mse')
    return model

### 1.4 Processing Subsets

In [23]:
def process_subset(sub_length, df):
    print(f"Running subset length: {sub_length}")
    # Subset the data for the current length
    data_subset = df.tail(sub_length)
    values = data_subset['values_scale'].values
    
    # Check if we have enough data points
    if len(values) < 14:  # Minimum required for 12 steps and 2 future steps
        print("Not enough data points for this subset length.")
        return None
    
    # Prepare the data for LSTM
    X, y = create_sequences(values, n_steps=12, n_future=2)
    
    # Build and train the LSTM model
    model = build_lstm_model(input_shape=(X.shape[1], X.shape[2]))
    history = model.fit(X, y, epochs=50, verbose=0)
    
    # Make predictions (two-step ahead)
    y_pred = model.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_pred - y[-1])**2))
    
    # Prepare forecast dates
    last_date = data_subset['time'].iloc[-1]
    future_dates = [last_date + pd.DateOffset(months=i+1) for i in range(2)]
    
    # Store results
    results = {
        'sub_length': sub_length,
        'rmse': rmse,
        'predictions': y_pred.flatten(),
        'actual': y[-1].flatten(),
        'forecast_dates': future_dates
    }
    return results

### 1.5. Main estimation function

In [19]:
# 5. Main Estimation Function
def estimate_lstm_models(data, country, series, sub=6):
    df = data[data['geo'] == country].copy()
    df.reset_index(drop=True, inplace=True) 

    df = df[['time', 'geo', series]].dropna()  # Drop rows with NaN in 'series'
    df['values'] = df[series]
    
    # Check if we have enough data after dropping NaNs
    if df.empty or len(df) < 14:
        print("Not enough data after dropping missing values.")
        return pd.DataFrame()
    
    df = clean_data(df)
    
    # Sort by time
    df = df.sort_values('time').reset_index(drop=True)
    
    # Define the lengths of subsets you want to test
    total_length = len(df)
    sub_lengths = [6 * i for i in range(1, (total_length // 6) + 1)]
    
    results_list = []
    for sub_length in sub_lengths:
        result = process_subset(sub_length, df)
        if result is not None:
            result.update({'geo': country, 'series': series})
            results_list.append(result)
    
    if results_list:
        results_df = pd.DataFrame(results_list)
        return results_df
    else:
        print("No valid results to display.")
        return pd.DataFrame()

## 2. First estimation

In [14]:
file_path = '../data_ignore/df_2024-10.csv'  # Replace with your actual file path
# Specify the country and series you want to analyze
country = 'EU27_2020'    # Replace with the actual country code from your data
series = 'values'        # Column name of your series (already 'values' in your data)

# Load the data
data = load_data(file_path)

data.head()

,freq,stk_flow,unit,partner,indic,geo,time,values
0,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,EU27_2020,1999-01-01,NaN
1,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,EU27_2020,1999-02-01,NaN
2,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,EU27_2020,1999-03-01,NaN
3,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,EU27_2020,1999-04-01,NaN
4,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,EU27_2020,1999-05-01,NaN


In [18]:
df = data[data['geo'] == country].copy()
df.reset_index(drop=True, inplace=True) 

df = df[['time', 'geo', series]].dropna()  # Drop rows with NaN in 'series'
df['values'] = df[series]
df.head()

,time,geo,values
36,2002-01-01,EU27_2020,75921.4
37,2002-02-01,EU27_2020,78349.3
38,2002-03-01,EU27_2020,86735.4
39,2002-04-01,EU27_2020,84412.5
40,2002-05-01,EU27_2020,85247.8


In [24]:
file_path = '../data_ignore/df_2024-10.csv'  # Replace with your actual file path

# Load the data
data = load_data(file_path)

# Specify the country and series you want to analyze
country = 'EU27_2020'    # Replace with the actual country code from your data
series = 'values'        # Column name of your series (already 'values' in your data)

# Estimate LSTM models and get results
results = estimate_lstm_models(data, country, series)

# Check if results are not empty
if not results.empty:
    # Display the results
    print(results)
    
    # Optional: Save results to CSV
    results.to_csv('lstm_results.csv', index=False)
else:
    print("No results to display.")

Running subset length: 6
Not enough data points for this subset length.
Running subset length: 12
Not enough data points for this subset length.
Running subset length: 18


2024-11-18 13:31:50.027083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 102ms/step
Running subset length: 24


2024-11-18 13:31:57.531304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-18 13:31:57.893560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 104ms/step
Running subset length: 30


...

2024-11-18 13:54:39.480061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-18 13:54:39.844470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 111ms/step
Running subset length: 270


2024-11-18 13:55:37.960244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-18 13:55:38.324605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 111ms/step
    sub_length      rmse             predictions  \
0           18  0.422655  [2.0088964, 1.8794075]   
1           24  0.413351  [1.8575772, 1.8193538]   
2           30  0.400159   [1.730489, 1.7143621]   
3           36  0.364762  [1.8328775, 1.7281061]   
4           42  0.437661  [1.9043366, 1.8751353]   
5           48  0.398240  [1.6926188, 1.6738517]   
6           54  0.418953  [1.8186469, 1.8085024]   
7           60  0.419641  [1.7797867, 1.7863196]   
8           66  0.405145    [1.76516, 1.7512678]   
9           72  0.416182  [1.7771593, 1.7787231]   
10          78  0.354698   [1.6914319, 1.574486]   
11          84  0.422837  [1.7842426, 1.7945567]   
12          90  0.399892  [1.7589895, 1.7373347]   
13          96  0.403441  [1.7551901, 1.7407423]   
14         102  0.398732  [1.7655995, 1.7402997]   
15         108  0.413740  [1.7861292, 1.7804894]   
16         114  0.375899  [1.8149216, 1.7353779]   
17         

2024-11-18 13:56:43.712492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [33]:
# Find the minimum RMSE value
min_rmse_value = results['rmse'].min()

# Get all rows with the minimum RMSE
results[results['rmse'] == min_rmse_value]

,sub_length,rmse,predictions,actual,forecast_dates,geo,series
27,180,0.325061,"[1.8055999, 1.6389209]","[2.1041043859678243, 1.2893140393353182]","[2024-09-01 00:00:00, 2024-10-01 00:00:00]",EU27_2020,values


In [35]:
df.tail()

,geo,time,values,values_scale,values_log,values_scale_mean,values_scale_sd
267,EU27_2020,2024-04-01,222460.1,1.939688,12.312503,142247.045221,41429.810588
268,EU27_2020,2024-05-01,216901.6,1.805274,12.287199,142247.045221,41429.810588
269,EU27_2020,2024-06-01,213480.9,1.722556,12.271303,142247.045221,41429.810588
270,EU27_2020,2024-07-01,229259.3,2.104104,12.342609,142247.045221,41429.810588
271,EU27_2020,2024-08-01,195564.8,1.289314,12.183647,142247.045221,41429.810588


In [38]:
data.tail()

,freq,stk_flow,unit,partner,indic,geo,time,values
8619,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,SE,2024-04-01,7283.1
8620,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,SE,2024-05-01,7749.4
8621,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,SE,2024-06-01,7086.6
8622,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,SE,2024-07-01,6549.0
8623,M,EXP,MIO-EUR-NSA,EXT_EU27_2020,ET-T,SE,2024-08-01,6122.8


**IMPORTANT**:

Need to review code such that the date for the predictions is set correct. 